In [ ]:
import requests
import json
import pandas as pd

# Zoho OAuth credentials and settings
client_id = '1000.ASZ7UO38XJWD1IDZOPB6EOEC3NIUCW'
client_secret = 'b6dd7c3bc0d313249fe809151e05ce2a9f099ec762'
scope = 'ZohoBooks.fullaccess.all,ZohoInventory.fullaccess.all'
token_url = "https://accounts.zoho.in/oauth/v2/token"
refresh_token_file = 'refresh_token.json'
access_token_file = 'access_token.json'

def get_refresh_token():
    try:
        with open(refresh_token_file, 'r') as f:
            refresh_token = json.load(f).get('refresh_token')
            if not refresh_token:
                print("Refresh token not found in the file.")
                return None
            return refresh_token
    except FileNotFoundError:
        print("Refresh token file not found.")
        return None

def save_access_token(access_token):
    data = {'access_token': access_token}
    with open(access_token_file, 'w') as f:
        json.dump(data, f)
    print(f"Access token saved to {access_token_file}")

def refresh_access_token(refresh_token):
    data = {
        'grant_type': 'refresh_token',
        'client_id': client_id,
        'client_secret': client_secret,
        'refresh_token': refresh_token
    }
    response = requests.post(token_url, data=data)
    if response.status_code == 200:
        tokens = response.json()
        if 'access_token' in tokens:
            access_token = tokens['access_token']
            save_access_token(access_token)  # Save access token to a file
            return access_token
        else:
            print("Failed to retrieve access token. Response:", tokens)
            return None
    else:
        print(f"Failed to refresh access token: {response.status_code} - {response.text}")
        return None

def fetch_data(url, headers, file_name):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        content_type = response.headers.get('Content-Type')
        if 'application/json' in content_type:
            json_data = response.json()
            if 'inventory_valuation' in json_data:
                df = pd.json_normalize(json_data['inventory_valuation'])
            else:
                print("JSON data does not contain 'inventory_valuation'")
                return False
            df.to_excel(file_name, index=False)
        elif 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' in content_type or 'text/xlsx' in content_type:
            with open(file_name, 'wb') as file:
                file.write(response.content)
        else:
            print(f"Unexpected content type: {content_type}")
            return False
        print(f'Data has been saved to {file_name}')
        return True
    else:
        print(f'Failed to fetch data: {response.status_code} - {response.text}')
        return False

def main():
    refresh_token = get_refresh_token()
    if refresh_token:
        access_token = refresh_access_token(refresh_token)
        if access_token:
            url = 'https://books.zoho.in/api/v3/export?entity=salesorder&accept=xlsx&async=false&filter_by=OrderDate.CustomDate&from_date=2024-06-01&to_date=2024-06-30&Status.All&sort_column=created_time&sort_order=D&can_export_pii_fields=false&export_template_id=&decimal_format=.&x-zb-source=zbclient&organization_id=60026691546&frameorigin=https%3A%2F%2Fbooks.zoho.in'
            headers = {'Authorization': f'Zoho-oauthtoken {access_token}'}
            fetch_data(url, headers, 'inventory.xlsx')
        else:
            print("Failed to obtain access token. Exiting.")
    else:
        print("No refresh token found. Authorization required.")

if __name__ == '__main__':
    main()